### Convert df_events to meta dataframe for pyphotometry analysis
- Meant to be a bridge between the pyphotometry analysis and df_events for the time being 

In [2]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import seaborn as sns
import numpy as np
from trialexp.process.pycontrol.session_analysis import *
from trialexp.process.pycontrol.utils import *

### Read task definition

In [3]:
basefolder = r'C:\code\trialexp'
tasks = pd.read_csv(basefolder+'\\params\\tasks_params.csv', usecols=[1, 2, 3, 4], index_col=False)

In [4]:
from trialexp.process.pycontrol.session_analysis import Session

In [5]:
conditions, triggers, events_to_process = Session.get_task_specs(tasks, 'reaching_go_spout_incr_break2_nov22')

# # extract condition from the event list
# df_conditions = df_events[df_events.event_name.isin(conditions)]
# df_conditions.set_index('time', inplace=True, drop=True)
# df_conditions

`df_condition` is nothing more than a dataframe containing the timestamp and name of the condition as defined in the task file. It also contains, the name of the name of the trigger event and trial number

In [3]:
from trialexp.process.pycontrol.session_analysis import Session

pycontrol_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\'
filename = 'kms062-2023-02-06-112451'
smrx_output_dir = 'Z:\\Teris\\ASAP\\pycontrol\\reaching_go_spout_nov22\\processed\\'

session = Session(pycontrol_dir+filename+'.txt')
df_events = parse_events(session)
# df_events = pd.read_pickle('df_events.pkl')
df_events

,state,event_name,time
1,short_break,state_change,0
2,short_break,bar_off,345
3,short_break,bar,425
4,short_break,bar_off,795
5,short_break,bar,1292
...,...,...,...
550,busy_win,bar,201003
551,busy_win,bar_off,201085
552,busy_win,bar,201102
553,busy_win,bar_off,201118


In [5]:
trial_window = [-2000, 4000]

conditions, triggers, events_to_process = Session.get_task_specs(tasks, 'reaching_go_spout_incr_break2_nov22')

df_events_trials, df_events = extract_trial_by_trigger(df_events, triggers[0], conditions+events_to_process, 
                                            trial_window, 'test', session.datetime)

NameError: name 'tasks' is not defined

In [85]:
df_conditions = compute_conditions_by_trial(df_events_trials, conditions)
df_conditions

,uid,trigger,valid,spout,US_end_timer
trial_nb,,,,,
1.0,test_2023-02-06_11:24:51_1.0,busy_win,True,True,True
2.0,test_2023-02-06_11:24:51_2.0,busy_win,True,True,True
3.0,test_2023-02-06_11:24:51_3.0,busy_win,True,True,True
4.0,test_2023-02-06_11:24:51_4.0,busy_win,True,True,True
5.0,test_2023-02-06_11:24:51_5.0,busy_win,True,True,True
6.0,test_2023-02-06_11:24:51_6.0,busy_win,True,True,True
7.0,test_2023-02-06_11:24:51_7.0,busy_win,True,True,True
8.0,test_2023-02-06_11:24:51_8.0,busy_win,True,True,True
9.0,test_2023-02-06_11:24:51_9.0,busy_win,True,True,True
